In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1) # sets the seed for generating the random numbers

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5) # 2 words in vocab, 5 dimensional embedding
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-2.9718  1.7070 -0.4305 -2.2820  0.5237
[torch.FloatTensor of size 1x5]



In [12]:
# NGram Sequential Language Model
# Compute Loss function on some examples and update parameters with backpropagation
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# build a list of tuples, each tuple is ([word_i-2, word_i-1], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
print(trigrams[:3])
vocab = set(test_sentence)

# build the indexed vocabulary
word_to_ix = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs
    
losses = [] 
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(100):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        # step 1. prepare the input to passed through the model
        # turn words into integer and wrap them in variables
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        # step 2. torch accumulates the gradients, before passing in a new 
        # instance zero out the gradients from old instance
        model.zero_grad()
        
        # step 3. run the forward pass to get log probabilities over next words
        log_probs = model(context_var)
        
        # step 4. compute the loss fn. 
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # step 5. do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
    
print(losses)
 

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[
 521.8846
[torch.FloatTensor of size 1]
, 
 519.5397
[torch.FloatTensor of size 1]
, 
 517.2101
[torch.FloatTensor of size 1]
, 
 514.8952
[torch.FloatTensor of size 1]
, 
 512.5935
[torch.FloatTensor of size 1]
, 
 510.3072
[torch.FloatTensor of size 1]
, 
 508.0348
[torch.FloatTensor of size 1]
, 
 505.7744
[torch.FloatTensor of size 1]
, 
 503.5252
[torch.FloatTensor of size 1]
, 
 501.2857
[torch.FloatTensor of size 1]
, 
 499.0553
[torch.FloatTensor of size 1]
, 
 496.8347
[torch.FloatTensor of size 1]
, 
 494.6233
[torch.FloatTensor of size 1]
, 
 492.4202
[torch.FloatTensor of size 1]
, 
 490.2253
[torch.FloatTensor of size 1]
, 
 488.0381
[torch.FloatTensor of size 1]
, 
 485.8571
[torch.FloatTensor of size 1]
, 
 483.6800
[torch.FloatTensor of size 1]
, 
 481.5065
[torch.FloatTensor of size 1]
, 
 479.3361
[torch.FloatTensor of size 1]
, 
 477.1702
[torch.FloatTensor of size 

In [19]:
# Continuous Bag of Words (CBOW)
CONTEXT_SIZE = 2 # Window size x words to the left, x to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1], raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

print(data[:5]) # print out some training examples

class CBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # nn.Linear(x, y) -> weight matrix y X x 
        # linear(input) -> input.matmul(weight.t())
        self.linear1 = nn.Linear(embedding_dim * context_size, 128)
        self.linear2 = nn.Linear(128, vocab_size)
     
    def forward(self, inputs):
        embeds = self.embedding(inputs).view(1, -1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs
    
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

make_context_vector(data[0][0], word_to_ix)

losses = []
model = CBOW(len(word_to_ix), 10, 4)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        model.zero_grad()
        context_vars = make_context_vector(context, word_to_ix)
        log_probs = model(context_vars)
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)

print(losses)
print()

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
[
 230.0602
[torch.FloatTensor of size 1]
, 
 228.6984
[torch.FloatTensor of size 1]
, 
 227.3483
[torch.FloatTensor of size 1]
, 
 226.0088
[torch.FloatTensor of size 1]
, 
 224.6777
[torch.FloatTensor of size 1]
, 
 223.3547
[torch.FloatTensor of size 1]
, 
 222.0405
[torch.FloatTensor of size 1]
, 
 220.7339
[torch.FloatTensor of size 1]
, 
 219.4350
[torch.FloatTensor of size 1]
, 
 218.1425
[torch.FloatTensor of size 1]
]
<generator object parameters at 0x1118b2a50>
